In [ ]:
import pandas as pd
import glob
import os
import numpy as np
from datetime import datetime

# Creating dfs

In [ ]:
path = r"L:/LovbeskyttetMapper01/HjertekirurgiDTU/Gruppe_1_DLMV/DO_NOT_TOUCH/"
all_files = glob.glob(os.path.join(path, "*.csv"))
dataframes = {}

for file in all_files:
    file_name = os.path.splitext(os.path.basename(file))[0]
    dataframes[file_name] = pd.read_csv(file, index_col = 'IDno')

In [ ]:
alco_smoking = pd.read_csv("L:/LovbeskyttetMapper01/HjertekirurgiDTU/Gruppe_1_DLMV/DO_NOT_TOUCH/Alkohol og rygning.csv", index_col='IDno')

In [ ]:
ADT = dataframes["ADT"]
haemodynamic = dataframes["Advancerede hæmodynamiske mål"]
alco_smoking = dataframes["Alkohol og rygning"]
all_operation_1_week_after_first = dataframes["Alle operationer lavet en uge efter den føste operation"]
anaestesidata = dataframes["Anæstesidata"]
anaestesi_raw = dataframes["Anæstesihændelse"]
anaestesi = anaestesi = pd.read_csv("L:/LovbeskyttetMapper01/HjertekirurgiDTU/Gruppe_1_DLMV/anaestesi.csv", index_col='IDno')
bleeding = dataframes["Blødning"]
dialyse = dataframes["Dialyse"]
EKG = dataframes["EKG-data"]
ekko = dataframes["Ekkokardiografi"]
height = dataframes["Højde"]
ITA = dataframes["ITA"]
KAG = dataframes["KAG data"]
lab = dataframes["Lab svar"]
pleuradraen = dataframes["Pleuradræn"]
population = dataframes["Populationen"]
problem = dataframes["Problemliste"]
respirator = dataframes["Respirator data"]
total_excretion = dataframes["Samlet udskillelse"]
spiromitri = dataframes["Spiromitri"]
total_incretion = dataframes["Total indgift"]
urin = dataframes["Urin"]
weight = dataframes["Vægt"]
vitale = dataframes["cirk. vitale værdier"]
intellispace = dataframes["intellispace"]
medicin = dataframes["medicin"]

In [ ]:
death_year = pd.read_csv("L:/LovbeskyttetMapper01/HjertekirurgiDTU/Gruppe_1_DLMV/death_year", index_col='IDno')

# Converting to datetime

In [ ]:
population_ = population.copy()
population_['Operationsdato'] = population_['Operationsdato'].apply(lambda x: datetime.strptime(x.rstrip('Z'), '%Y-%m-%dT%H:%M:%S'))
population_ = population_.dropna(subset = ['Indlæggelsestidspunkt', 'Udskrivelsestidspunkt'])
population_['Indlæggelsestidspunkt'] = population_['Indlæggelsestidspunkt'].apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S.%f'))
population_['Udskrivelsestidspunkt'] = population_['Udskrivelsestidspunkt'].apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S.%f'))
population_.loc[population_['Dødsdato'].notnull(), 'Dødsdato'] = population_.loc[population_['Dødsdato'].notnull(), 'Dødsdato'].apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S.%f'))

population = population_
del population_

## Defining operation start and finish

In [ ]:
X = anaestesi.groupby(level=0).filter(lambda x: (x['Hændelse'] == 'Induktion').any())
X = anaestesi.groupby(level=0).filter(lambda x: (x['Hændelse'] == 'Stop Data Indsamling').any())

In [ ]:
a = X.groupby(level=0, sort=False)['Hændelse'].apply(lambda x: x.duplicated(keep='first'))
b = X.groupby(level=0, sort=False)['Hændelse'].apply(lambda x: x == 'Induktion')
mask_ind = a & b
mask_ind = [not b for b in mask_ind]

In [ ]:
X_ = X[mask_ind]

In [ ]:
c = X_.groupby(level=0, sort=False)['Hændelse'].apply(lambda x: x.duplicated(keep='first'))
d = X_.groupby(level=0, sort=False)['Hændelse'].apply(lambda x: x == 'Stop Data Indsamling')
mask_stop = c&d
mask_stop = [not b for b in mask_stop]

In [ ]:
X = X_[mask_stop]

In [ ]:
X = X.groupby(level=0).filter(lambda x: (x['Hændelse'] == 'Induktion').any())
X = X.groupby(level=0).filter(lambda x: (x['Hændelse'] == 'Stop Data Indsamling').any())

In [ ]:
X_ = X.groupby(level=0, sort= False).apply(lambda x: x['Hændelsestidspunkt'] <= x[x['Hændelse'] == 'Stop Data Indsamling']['Hændelsestidspunkt'][0])

In [ ]:
X = X.groupby(level=0).filter(lambda x: ((x['Hændelse'] == 'Aorta tang på').any()))
X = X.groupby(level=0).filter(lambda x: (x['Hændelse'] == 'Aorta tang af').any())

In [ ]:
anaestesi = X

In [ ]:
anaestesi = anaestesi.groupby(level=0).filter(lambda x: ((x['Hændelse'] == 'Induktion').any()))


# Population

In [ ]:
anaestesi = anaestesi = pd.read_csv("L:/LovbeskyttetMapper01/HjertekirurgiDTU/Gruppe_1_DLMV/anaestesi.csv", index_col='IDno')

bool = []
for i in population.index:
    bool.append(i in anaestesi.index.unique())

In [ ]:
population = pd.read_csv("L:/LovbeskyttetMapper01/HjertekirurgiDTU/Gruppe_1_DLMV/DO_NOT_TOUCH/Populationen.csv", index_col='IDno')

In [ ]:
population['Operationsdato'] = population['Operationsdato'].apply(lambda x: datetime.strptime(x.rstrip('Z'), '%Y-%m-%dT%H:%M:%S'))

In [ ]:
bool = []
for i in population.index:
    bool.append(i in anaestesi.index.unique())

In [ ]:
age = population[bool]['Alder på operationstidspunkt'].groupby(level=0, sort=False).apply(lambda x: x.drop_duplicates(keep ='first'))
plc= age.reset_index(0)
plc.index = plc.index.rename('asdf')
plc = plc.reset_index(0)
plc = plc.drop(columns=['asdf'])
plc.index = plc['IDno']
age = plc.drop(columns='IDno')

In [ ]:
age = age.reset_index().drop_duplicates(subset='IDno', keep='first').set_index('IDno')

In [ ]:
death_year = population[bool]['Død inden for 1 år af operation'].groupby(level=0, sort=False).apply(lambda x: x.drop_duplicates(keep ='first'))
plc= death_year.reset_index(0)
plc.index = plc.index.rename('asdf')
plc = plc.reset_index(0)
plc = plc.drop(columns=['asdf'])
plc.index = plc['IDno']
death_year= plc.drop(columns='IDno')

In [ ]:
death_year.insert(1,'age',age)

In [ ]:
sex = population[bool]['Køn'][~population[bool]['Køn'].index.duplicated(keep='last')]
death_year.insert(2, 'sex', sex)

In [ ]:
# Turn strings into numbers and if we see interval, convert it to a single number by calculating the mean

def sorting_intervals(genstande):
    if isinstance(genstande, str):
        if "-" in genstande:
            lower, upper = map(int, genstande.split('-'))
            return round((lower + upper) / 2)
        else:
            return int(genstande)
    elif isinstance(genstande, (int, float)):
        return genstande
    else:
        return None
    
alco_smoking['Genstande per uge'] = round(alco_smoking['Genstande per uge'].apply(sorting_intervals))
alco_smoking['Genstande per uge'].unique()

In [ ]:
alco_smoking = alco_smoking[alco_smoking['Rygning'] != 'Aldrig vurderet']
alco_smoking = alco_smoking[alco_smoking['Rygning'] != 'Ukendt']

In [ ]:
alco_smoking = alco_smoking.replace('Ryger, aktuel status ukendt', 'Ryger').replace('Hver dag', 'Ryger').replace('Lejlighedsvis ryger', 'Nogle dage').replace('Udsat for passiv rygning - aldrig været ryger', 'Passiv rygning')

In [ ]:
alco_smoking = alco_smoking[~alco_smoking.index.duplicated(keep='last')]

In [ ]:
fil = [id in list(death_year.index) for id in alco_smoking.index]

In [ ]:
alco_smoking = alco_smoking[fil]

In [ ]:
problem = pd.read_csv("L:/LovbeskyttetMapper01/HjertekirurgiDTU/Gruppe_1_DLMV/DO_NOT_TOUCH/Problemliste.csv", index_col='IDno')

In [ ]:
fil = [id in death_year.index for id in problem.index]
problem = problem[fil]

In [ ]:
f = []
for d in problem['Diagnose SKS']:
    if 'DJ' in str(d):
        f.append(True)
sum(f)

In [ ]:
filter = []
for d in problem['Diagnose SKS']:
    if 'DJ4' in str(d):
        filter.append(True)
    elif 'DJ12' in str(d):
        filter.append(True)
    elif 'DJ13' in str(d):
        filter.append(True)
    elif 'DJ14' in str(d):
        filter.append(True)
    elif 'DJ15' in str(d):
        filter.append(True)
    elif 'DJ16' in str(d):
        filter.append(True)
    elif 'DJ17' in str(d):
        filter.append(True)
    elif 'DJ6' in str(d):
        filter.append(True)
    elif 'DJ7' in str(d):
        filter.append(True)
    else:
        filter.append(False)


In [ ]:
lung_disease_filter = []

In [ ]:
resp_disease_filter = []
for d in problem['Diagnose SKS']:
    if 'DJ' in str(d):
        resp_disease_filter.append(True)
    else:
        resp_disease_filter.append(False)

circ_disease_filter = []
for d in problem['Diagnose SKS']:
    if 'DI' in str(d):
        circ_disease_filter.append(True)
    else: 
        circ_disease_filter.append(False)

In [ ]:
resp_dis = [idx in problem[resp_disease_filter].index.unique() for idx in death_year.index]
circ = [idx in problem[circ_disease_filter].index.unique() for idx in death_year.index]

In [ ]:
death_year.insert(3, 'Respiratory Disease', resp_dis)

In [ ]:
death_year.insert(4, 'Circulatory Organ Disease', circ)

In [ ]:
death_year = death_year.replace('Nej', False).replace('Ja',True)

In [ ]:
weight = pd.read_csv("L:/LovbeskyttetMapper01/HjertekirurgiDTU/Gruppe_1_DLMV/Vægt.csv", index_col='IDno')

In [ ]:
w_filt = [idx in anaestesi.index.unique() for idx in weight.index]

In [ ]:
weight = weight[w_filt]

In [ ]:
weight['Målingstidspunkt'] = weight['Målingstidspunkt'].apply(lambda x: datetime.strptime(str(x).rstrip('Z'), '%Y-%m-%dT%H:%M:%S'))

In [ ]:
X = anaestesi[anaestesi['Hændelse'] == 'Stop Data Indsamling']


In [ ]:
weight.groupby(level=0, sort = False)['Målingstidspunkt']

In [ ]:
weightavg = (weight['Vægt'].groupby(level=0).head(3)).groupby(level = 0, sort = False).mean()


In [ ]:
death_year = death_year.merge(weightavg, how='left', left_index = True, right_index = True)

In [ ]:
height = pd.read_csv("L:/LovbeskyttetMapper01/HjertekirurgiDTU/Gruppe_1_DLMV/Højde.csv", index_col='IDno', sep=';')

In [ ]:
height['Højde'] = height['Højde'].apply(lambda x: float(x.replace(',', '.')))

In [ ]:
avg_height = height['Højde'].groupby(level=0, sort = False).mean()

In [ ]:
death_year = death_year.merge(avg_height, how='left', left_index=True, right_index=True)

In [ ]:
spiromitri = pd.read_csv("L:/LovbeskyttetMapper01/HjertekirurgiDTU/Gruppe_1_DLMV/Spiromitri.csv", index_col='IDno', sep=';')


In [ ]:
FVC_mask = ~spiromitri[spiromitri['Prøve'] == 'FVC (%)'].index.duplicated()
spiromitri[spiromitri['Prøve'] == 'FVC (%)'][FVC_mask]

In [ ]:
FEV_mask = ~spiromitri[spiromitri['Prøve'] == 'FEV 1 / FVC'].index.duplicated()
spiromitri[spiromitri['Prøve'] == 'FEV 1 / FVC'][FEV_mask]

In [ ]:
FEV = spiromitri[spiromitri['Prøve'] == 'FEV 1 / FVC'][FEV_mask]

In [ ]:
for i in FEV.index:
    j = FEV['Værdi'].loc[i]

    if isinstance(j, str):
        j = j.strip()
        if '0.' in j:
            FEV.at[i, 'Værdi'] = float(j) * 100
        
        elif 'n' in j or j == '%' or j == '63/63':
            FEV.drop(i, inplace = True)
        
        elif '%' in j:
            j = j.replace('%', '').replace(',','.')
            FEV.at[i, 'Værdi'] = float(j) / 100

        elif ',' in j:
            j = j.replace(',','.')
            FEV.at[i, 'Værdi'] = float(j)        
    else:
        FEV.at[i, 'Værdi'] = float(j)

for i in FEV.index:
    j = FEV['Værdi'].loc[i]

    if float(j) > 1000:
        FEV.at[i, 'Værdi'] = float(j) / 100

In [ ]:
obstruction_filter = []
for procent in FEV['Værdi'].index:
    if float(FEV['Værdi'].loc[procent]) >= 70:
        obstruction_filter.append(False)
    else:
        obstruction_filter.append(True)

In [ ]:
death_year = death_year.merge(FEV['Værdi'], how='left', left_index=True, right_index=True)

In [ ]:
death_year = death_year.rename(columns={'Genital or urine related diseases':'Genital or urine related diseases'})
death_year = death_year.rename(columns={'Værdi':'Lung obstruction'})


In [ ]:
type1_dia_filter = []
for d in problem['Diagnose SKS']:
    if 'DE10' in str(d):
        type1_dia_filter.append(True)
    else:
        type1_dia_filter.append(False)

type2_dia_filter = []
for d in problem['Diagnose SKS']:
    if 'DE11' in str(d):
        type2_dia_filter.append(True)
    else:
        type2_dia_filter.append(False)

In [ ]:
type1_dia = [idx in problem[type1_dia_filter].index.unique() for idx in death_year.index]
type2_dia = [idx in problem[type2_dia_filter].index.unique() for idx in death_year.index]


In [ ]:
disease_filter = []
for d in problem['Diagnose SKS']:
    if 'DE' in str(d) and 'DE10' not in str(d) and 'DE11' not in str(d):
        disease_filter.append(True)
    else:
        disease_filter.append(False)


In [ ]:
disease = [idx in problem[disease_filter].index.unique() for idx in death_year.index]

In [ ]:
death_year.insert(5, 'Type 1 Diabetes', type1_dia)
death_year.insert(6, 'Type 2 Diabetes', type2_dia)

In [ ]:
death_year.insert(7, 'Other metabolic diseases', disease)

In [ ]:
genital_urine_fil = []
for d in problem['Diagnose SKS']:
    if 'DN' in str(d):
        genital_urine_fil.append(True)
    else:
        genital_urine_fil.append(False)

genital_urine = [idx in problem[genital_urine_fil].index.unique() for idx in death_year.index]



In [ ]:
death_year.insert(8, 'Geietal or urine related diseases', genital_urine)

In [ ]:
death_year.rename(columns={'Geietal or urine related diseases':'Genital or urine related diseases'}, inplace=True)

In [ ]:
population.merge(anaestesi, how='right', left_index=True, right_index=True)

In [ ]:
population['Operation'] = population['Operation SKS kode'].apply(lambda x: x[:3])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
death_year.to_csv('death_year')

In [ ]:
KFN = []
KFX = []
KFK = []
KFM = []
KFF = []
KFP = []
KFC = []
KFW = []
KFJ = []

for d in population['Operation']:
    if str(d) == 'KFN':
        KFN.append(True)
        KFX.append(False)
        KFK.append(False)
        KFM.append(False)
        KFF.append(False)
        KFP.append(False)
        KFC.append(False)
        KFW.append(False)
        KFJ.append(False)
    elif str(d) == 'KFX':
        KFN.append(False)
        KFX.append(True)
        KFK.append(False)
        KFM.append(False)
        KFF.append(False)
        KFP.append(False)
        KFC.append(False)
        KFW.append(False)
        KFJ.append(False)
    elif str(d) == 'KFK':
        KFN.append(False)
        KFX.append(False)
        KFK.append(True)
        KFM.append(False)
        KFF.append(False)
        KFP.append(False)
        KFC.append(False)
        KFW.append(False)
        KFJ.append(False)
    elif str(d) == 'KFM':
        KFN.append(False)
        KFX.append(False)
        KFK.append(False)
        KFM.append(True)
        KFF.append(False)
        KFP.append(False)
        KFC.append(False)
        KFW.append(False)
        KFJ.append(False)
    elif str(d) == 'KFF':
        KFN.append(False)
        KFX.append(False)
        KFK.append(False)
        KFM.append(False)
        KFF.append(True)
        KFP.append(False)
        KFC.append(False)
        KFW.append(False)
        KFJ.append(False)
    elif str(d) == 'KFP':
        KFN.append(False)
        KFX.append(False)
        KFK.append(False)
        KFM.append(False)
        KFF.append(False)
        KFP.append(True)
        KFC.append(False)
        KFW.append(False)
        KFJ.append(False)
    elif str(d) == 'KFC':
        KFN.append(False)
        KFX.append(False)
        KFK.append(False)
        KFM.append(False)
        KFF.append(False)
        KFP.append(False)
        KFC.append(True)
        KFW.append(False)
        KFJ.append(False)
    elif str(d) == 'KFW':
        KFN.append(False)
        KFX.append(False)
        KFK.append(False)
        KFM.append(False)
        KFF.append(False)
        KFP.append(False)
        KFC.append(False)
        KFW.append(True)
        KFJ.append(False)
    elif str(d) == 'KFJ':
        KFN.append(False)
        KFX.append(False)
        KFK.append(False)
        KFM.append(False)
        KFF.append(False)
        KFP.append(False)
        KFC.append(False)
        KFW.append(False)
        KFJ.append(True)
    else:
        KFN.append(False)
        KFX.append(False)
        KFK.append(False)
        KFM.append(False)
        KFF.append(False)
        KFP.append(False)
        KFC.append(False)
        KFW.append(False)
        KFJ.append(False)

In [ ]:
KFN_ = [idx in population[KFN].index.unique() for idx in death_year.index]
KFX_ = [idx in population[KFX].index.unique() for idx in death_year.index]
KFK_ = [idx in population[KFK].index.unique() for idx in death_year.index]
KFM_ = [idx in population[KFM].index.unique() for idx in death_year.index]
KFF_ = [idx in population[KFF].index.unique() for idx in death_year.index]
KFP_ = [idx in population[KFP].index.unique() for idx in death_year.index]
KFC_ = [idx in population[KFC].index.unique() for idx in death_year.index]
KFW_ = [idx in population[KFW].index.unique() for idx in death_year.index]
KFJ_ = [idx in population[KFJ].index.unique() for idx in death_year.index]


In [ ]:
death_year.insert(10, 'KFN', KFN_)

In [ ]:
death_year.insert(11, 'KFX', KFX_)
death_year.insert(12, 'KFK', KFK_)
death_year.insert(13, 'KFM', KFM_)
death_year.insert(14, 'KFF', KFF_)
death_year.insert(15, 'KFP', KFP_)
death_year.insert(16, 'KFC', KFC_)
death_year.insert(17, 'KFW', KFW_)
death_year.insert(18, 'KFJ', KFJ_)

In [ ]:
disease_filter = []
for d in problem['Diagnose SKS']:
    if 'DE' in str(d) and 'DE10' not in str(d) and 'DE11' not in str(d):
        disease_filter.append(True)
    else:
        disease_filter.append(False)


In [ ]:
other_op = []
for d in population['Operation']:
    if str(d) == 'KFH' or str(d)=='KFG' or str(d)=='KFE' or str(d)=='KFB' or str(d)=='KFQ' or str(d)=='KFD' or str(d)=='KFL' or str(d)=='KFA':
        other_op.append(True)
    else:
        other_op.append(False)


In [ ]:
other = [idx in population[other_op].index.unique() for idx in death_year.index]

In [ ]:
death_year.insert(8, 'Other operation', other)

In [ ]:
age = population[bool]['Alder på operationstidspunkt'].groupby(level=0, sort=False).apply(lambda x: x.drop_duplicates(keep ='first'))
plc= age.reset_index(0)
plc.index = plc.index.rename('asdf')
plc = plc.reset_index(0)
plc = plc.drop(columns=['asdf'])
plc.index = plc['IDno']
age = plc.drop(columns='IDno')
age

# Lab svar inkorporation

EGFR/1 (som er patientens nyefunktion), som er kreatanin men justeret for

Hæmoglobin (fokus fordi hvis tallet er for lavet = det er for farligt at lave operaiton)

Plastman glykose (også fokus = tegn på pre sukkersyge)

Ecosytter (også fokus = tegn på inflamation)

Tromposytter (ned prioriteteres = patienten har måske en kronisk påvirkning)

Troponin  (måske forkastes, fordi den har samme formål som Kreatinkrenase MB;P)

Kreatinkrenase MB;P (fokus = mistænkning om blodprop i hjertet jo højere jo værre)

**EFGR nyre**


In [ ]:
lab = pd.read_csv('Lab svar.csv', index_col='IDno')

In [ ]:
lab['V3'] = lab['V3'].apply(lambda x: datetime.strptime(x.rstrip('Z'), '%Y-%m-%dT%H:%M:%S'))

In [ ]:
lab = lab[lab.index.isin(death_year.index)]

In [ ]:
lab.rename(columns={'V3':'Time'}, inplace=True)
lab.rename(columns={'V2':'Type'}, inplace=True)
lab.rename(columns={'V4':'Value'}, inplace=True)

In [ ]:
lab_anae = lab.merge(anaestesi, left_index=True, right_index=True)

In [ ]:
lab_anae = lab_anae[lab_anae['Hændelse'] == 'Induktion']

In [ ]:
X = lab_anae[lab_anae['Time'] < lab_anae['Hændelsestidspunkt']]

In [ ]:
EFGR_nyre = X[X['Type'] == 'EGFR/1,73M²(CKD-EPI);NYRE']
EFGR_nyre = EFGR_nyre.rename(columns={'Unit':'Value'})

In [ ]:
EFGR_nyre['Value'] = pd.to_numeric(EFGR_nyre['Value'], errors='coerce')

In [ ]:
EFGR_nyre = EFGR_nyre.dropna(subset=['Value'])

In [ ]:
avg_EFGR = EFGR_nyre.groupby(level=0, sort=False).apply(lambda x: x['Value'].mean())
death_year.insert(9, 'Nyrefunktion', avg_EFGR)

**Hæmoglobin**

In [ ]:
haemo = X[X['Type'] == 'HÆMOGLOBIN;B']
haemo['Value'] = pd.to_numeric(haemo['Value'], errors='coerce')
haemo = haemo.dropna(subset=['Value'])
haemo = haemo.groupby(level=0, sort=False).apply(lambda x: x['Value'].mean())
death_year = death_year.assign(Hæmoglobin = haemo)

 **Plasma Glukose**

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
gluk = X[X['Type'] == 'GLUKOSE;P']
gluk['Value'] = pd.to_numeric(gluk['Value'], errors='coerce')
gluk = gluk.dropna(subset=['Value'])
gluk = gluk.groupby(level=0, sort=False).apply(lambda x: x['Value'].mean())
death_year = death_year.assign(Glucose = gluk)

**Kreatinkinase og kreatinekinase MB**

In [ ]:
kkmb = X[X['Type'] == 'KREATINKINASE MB;P']
kkmb['Value'] = pd.to_numeric(kkmb['Value'], errors='coerce')
kkmb = kkmb.dropna(subset=['Value'])
kkmb = kkmb.groupby(level=0, sort=False).apply(lambda x: x['Value'].mean())
death_year.insert(11, 'Kreatinkinase MB', kkmb)

**Leukocytter**

In [ ]:
leuk = X[X['Type'] == 'LEUKOCYTTER;B']['Value']
leuk = pd.to_numeric(leuk, errors='coerce')
leuk = leuk.dropna()
leuk = leuk.groupby(level=0, sort=False).apply(lambda x: x.mean())
death_year = death_year.assign(Leukocytter = leuk)

**Tromposytter**

In [ ]:
trom = X[X['Type'] == 'TROMBOCYTTER;B']['Value']
trom = pd.to_numeric(trom, errors='coerce')
trom = trom.dropna()
trom = trom.groupby(level=0, sort=False).apply(lambda x: x.mean())
death_year = death_year.assign(Trombocytter = trom)

# EKG

In [ ]:
ekg = pd.read_csv('EKG-data.csv', index_col='IDno')

In [ ]:
ekg['Tidspunkt'] = ekg['Tidspunkt'].apply(lambda x: datetime.strptime(x.rstrip('Z'), '%Y-%m-%dT%H:%M:%S'))

In [ ]:
ekg_anae = ekg.merge(anaestesi, left_index=True, right_index=True)
ekg_anae = ekg_anae[ekg_anae['Hændelse'] == 'Induktion']
ekg_anae = ekg_anae[ekg_anae['Tidspunkt'] < ekg_anae['Hændelsestidspunkt']]

In [ ]:
ekg_anae

In [ ]:
qtc_b = ekg_anae[ekg_anae['Navn'] == 'QTc (Bassett’s formel)']['Værdi']
qtc_b = pd.to_numeric(qtc, errors='coerce')
qtc_b = qtc_b.dropna()
qtc_b = qtc_b.groupby(level=0, sort=False).apply(lambda x: x.mean())
death_year.insert(13, 'QTc', qtc_b)

# KAG

In [ ]:
kag = pd.read_csv('KAG data.csv', sep=';', index_col='IDno')

In [ ]:
#Now i want to find the induction and the stop data collection for each patient that are closest to each other, also i want to only include patients who have aorta tang af and aorta tang på and aorta tang af
#I will do this by first filtering out all the inductions and then filtering out all the stop data collections
#Then I will filter out the ones that are closest to each other
#Then I will filter out the ones that have aorta tang af and aorta tang på and aorta tang af
anae = pd.read_csv('Anæstesidata.csv', index_col='IDno')
X = anae.groupby(level=0).filter(lambda x: (x['Hændelse'] == 'Induktion').any())
X = anae.groupby(level=0).filter(lambda x: (x['Hændelse'] == 'Stop Data Indsamling').any())
X = anae.groupby(level=0).filter(lambda x: (x['Hændelse'] == 'Aorta tang på').any())
X = anae.groupby(level=0).filter(lambda x: (x['Hændelse'] == 'Aorta tang af').any())


In [ ]:
#I will now eliminate duplicate inductions and stop data collections
a = X.groupby(level=0, sort=False)['Hændelse'].apply(lambda x: x.duplicated(keep='first'))
b = X.groupby(level=0, sort=False)['Hændelse'].apply(lambda x: x == 'Induktion')
mask_ind = a & b
mask_ind = [not b for b in mask_ind]
X_ = X[mask_ind]
c = X_.groupby(level=0, sort=False)['Hændelse'].apply(lambda x: x.duplicated(keep='first'))
d = X_.groupby(level=0, sort=False)['Hændelse'].apply(lambda x: x == 'Stop Data Indsamling')
mask_stop = c&d
mask_stop = [not b for b in mask_stop]
X_ = X_[mask_stop]
X = X_

In [ ]:
#I will now make sure nothing can come after stop data collection
X = X.groupby(level=0).filter(lambda x: (x['Hændelse'] == 'Induktion').any())
X = X.groupby(level=0).filter(lambda x: (x['Hændelse'] == 'Stop Data Indsamling').any())
X_ = X.groupby(level=0, sort= False).apply(lambda x: x['Hændelsestidspunkt'] <= x[x['Hændelse'] == 'Stop Data Indsamling']['Hændelsestidspunkt'][0])


In [ ]:
X_ = X_.reset_index(level = 0)
X_ = X_['Hændelsestidspunkt']

In [ ]:
X = X[X_]


In [ ]:
X = X.groupby(level=0).filter(lambda x: (x['Hændelse'] == 'Aorta tang på').any())
X = X.groupby(level=0).filter(lambda x: (x['Hændelse'] == 'Aorta tang af').any())


In [ ]:
death_year = death_year.dropna()

# death_year retouch (KØRES EFTER death_year bliver læst øverst oppe)

KØR BEGGE CELLER for har ikke gemt endnu

In [ ]:
death_year = pd.read_csv("L:/LovbeskyttetMapper01/HjertekirurgiDTU/Gruppe_1_DLMV/death_year", index_col='IDno')

In [ ]:
death_year = death_year.drop(columns='Højde')

Make all NaN to False and values under 70 to False, else True

In [ ]:
lung_ob = death_year['Lung obstruction']

death_year['Lung obstruction'] = lung_ob.apply(lambda x: False if x < 70 else True)
death_year.dropna()

# Standardizing datasættet

In [ ]:
from sklearn.preprocessing import StandardScaler

def standardize_scale(df, scale_cols):
    scaler = StandardScaler()
    for col in scale_cols:
        df[col] = scaler.fit_transform(df[col].values.reshape(-1, 1))
    return df

In [ ]:
scales = ['age', 'Vægt']
standardize_scale(death_year, scale_cols=scales)

In [ ]:
death_year = death_year.replace(True, 1).replace(False, 0)


# Perieoperative dataset

In [ ]:
perie_op = death_year.copy()

In [ ]:
lab = pd.read_csv('Lab svar.csv', index_col='IDno')


In [ ]:
c = [idx in death_year.index for idx in lab.index]

In [ ]:
lab_anae = lab.merge(anaestesi, left_index=True, right_index=True)
lab_anae = lab_anae[(lab_anae['Hændelse'] == 'Induktion') |( lab_anae['Hændelse'] == 'Stop Data Indsamling')]

In [ ]:
lab_anae = lab_anae.drop(columns = ['Hændelse', 'Hændelsestidspunkt'])


In [ ]:
lab_anae = lab_anae.drop_duplicates(keep = 'first')

In [ ]:
lab_anae['V3'] = lab_anae['V3'].apply(lambda x: datetime.strptime(x.rstrip('Z'), '%Y-%m-%dT%H:%M:%S'))

In [ ]:
startstop = {}
for idx in anaestesi.index.unique():
    start = anaestesi['Hændelsestidspunkt'][anaestesi['Hændelse'] == 'Induktion'].loc[idx]
    stop = anaestesi['Hændelsestidspunkt'][anaestesi['Hændelse'] == 'Stop Data Indsamling'].loc[idx]
    startstop[idx] = (start, stop)

In [ ]:
mask = lab_anae.groupby(level = 0, sort=False).apply(lambda x: (startstop[x.index[0]][1] > x['V3']) & (x['V3'] > startstop[x.index[0]][0]))

In [ ]:
perie_lab = lab_anae[mask.values]

In [ ]:
glukose_perie = perie_lab[perie_lab['V2'] == 'TROMBOCYTTER;B']

In [ ]:
perie_lab.groupby(level=0, sort=False).apply(lambda x: x['V3'].mean())

In [ ]:
ekg_anae = ekg.merge(anaestesi, left_index=True, right_index=True).drop(columns=['Hændelse', 'Hændelsestidspunkt'])
mask = ekg_anae.groupby(level = 0, sort=False).apply(lambda x: (startstop[x.index[0]][1] > x['Tidspunkt']) & (x['Tidspunkt'] > startstop[x.index[0]][0]))

In [ ]:
dia_anae = dialyse.merge(anaestesi, left_index=True, right_index=True).drop(columns=['Hændelse', 'Hændelsestidspunkt'])
mask = dia_anae.groupby(level = 0, sort=False).apply(lambda x: (startstop[x.index[0]][1] > x['Målingsdatotid']) & (x['Målingsdatotid'] > startstop[x.index[0]][0]))
                                                    

In [ ]:
urin = pd.read_csv('Urin.csv', index_col='IDno')

In [ ]:
urin = urin.merge(anaestesi, left_index=True, right_index=True).drop(columns=['Hændelse', 'Hændelsestidspunkt'])
urin['dato'] = urin['dato'].apply(lambda x: datetime.strptime(x.rstrip('Z'), '%Y-%m-%dT%H:%M:%S'))

In [ ]:
mask = urin.groupby(level = 0, sort=False).apply(lambda x: (startstop[x.index[0]][1] > x['dato']) & (x['dato'] > startstop[x.index[0]][0]))

In [ ]:
urin = urin[mask.values]

In [ ]:
urin = urin.drop_duplicates(subset='dato')

In [ ]:
urin_tot = urin.groupby(level = 0, sort=False).apply(lambda x: x['Urin (ml)'].sum()).to_frame()

In [ ]:
urin_tot.columns = ['Total urin ml (perioperativ)']


In [ ]:
perie_op = perie_op.merge(urin_tot, left_index=True, right_index=True)

In [ ]:
blood = pd.read_csv('Blødning.csv', index_col='IDno')
blood['dato'] = blood['dato'].apply(lambda x: datetime.strptime(x.rstrip('Z'), '%Y-%m-%dT%H:%M:%S'))
blood = blood.merge(anaestesi, left_index=True, right_index=True).drop(columns=['Hændelse', 'Hændelsestidspunkt'])
mask = blood.groupby(level = 0, sort=False).apply(lambda x: (startstop[x.index[0]][1] > x['dato']) & (x['dato'] > startstop[x.index[0]][0]))

In [ ]:
blood = blood.drop_duplicates(subset='dato' ,keep='first')

In [ ]:
blood_perie  = blood.groupby(level=0, sort=False).apply(lambda x: x['Blødning ml'].sum())
blood_perie = blood_perie.to_frame()
blood_perie.columns = ['Total blødning ml (perioperativ)']
perie_op = perie_op.merge(blood_perie, left_index=True, right_index=True)

In [ ]:
perie_lab[perie_lab['V2'] == 'ECV-BASE EXCESS;(POC)'].index.unique()

In [ ]:
cirk = pd.read_csv('cirk. vitale værdier.csv', index_col='IDno', on_bad_lines = 'skip', sep=';')
cirk = cirk.merge(anaestesi, left_index=True, right_index=True).drop(columns=['Hændelse', 'Hændelsestidspunkt'])

In [ ]:
cirk['Målingsdatotid'] = cirk['Målingsdatotid'].apply(lambda x: datetime.strptime(x.rstrip('Z'), '%Y-%m-%dT%H:%M:%S'))

In [ ]:
mask = cirk.groupby(level = 0, sort=False).apply(lambda x: (startstop[x.index[0]][1] > x['Målingsdatotid']) & (x['Målingsdatotid'] > startstop[x.index[0]][0]))

In [ ]:
cirk_perie = cirk[mask.values]

In [ ]:
puls_perie_avg = cirk_perie['Puls'].groupby(level=0, sort=False).apply(lambda x: x.mean()).to_frame(name='Puls AVG(perioperativ)')
puls_perie_sd = cirk_perie['Puls'].groupby(level=0, sort=False).apply(lambda x: x.std()).to_frame(name='Puls SD (perioperativ)')
saturation_perie_avg = cirk_perie['Saturation'].groupby(level=0, sort=False).apply(lambda x: x.mean()).to_frame(name='Saturation AVG(perioperativ)')
saturation_perie_sd = cirk_perie['Saturation'].groupby(level=0, sort=False).apply(lambda x: x.std()).to_frame(name='Saturation SD(perioperativ)')
perie_op = perie_op.merge(puls_perie_avg, left_index=True, right_index=True)
perie_op = perie_op.merge(puls_perie_sd, left_index=True, right_index=True)
perie_op = perie_op.merge(saturation_perie_avg, left_index=True, right_index=True)
perie_op = perie_op.merge(saturation_perie_sd, left_index=True, right_index=True)


# Post operative morbidity

In [ ]:
post_op = perie_op.copy()

In [ ]:
mask = lab_anae.groupby(level = 0, sort=False).apply(lambda x: (startstop[x.index[0]][1] < x['V3']))

In [ ]:
post_lab = lab_anae[mask.values]

In [ ]:
post_haemo = post_lab[post_lab['V2']=='HÆMOGLOBIN;B']
post_haemo['V4'] = pd.to_numeric(post_haemo['V4'], errors='coerce')
post_haemo = post_haemo.dropna(subset=['V4'])

In [ ]:
post_haemo_avg = post_haemo.groupby(level=0).apply(lambda x: x['V4'].mean())
post_haemo_avg = post_haemo_avg.to_frame(name='Hæmoglobin AVG (postoperative)')
post_haemo_sd = post_haemo.groupby(level=0).apply(lambda x: x['V4'].std())
post_haemo_sd = post_haemo_sd.to_frame(name='Hæmoglobin SD (postoperative)')
post_op = post_op.merge(post_haemo_avg, left_index=True, right_index=True)
post_op = post_op.merge(post_haemo_sd, left_index=True, right_index=True)

**Natrium**

In [ ]:
natrium_post = post_lab[post_lab['V2'] == 'NATRIUM;P']
natrium_post['V4'] = pd.to_numeric(natrium_post['V4'], errors='coerce')
natrium_post = natrium_post.dropna(subset=['V4'])
natrium_post_avg = natrium_post.groupby(level=0).apply(lambda x: x['V4'].mean())
natrium_post_avg = natrium_post_avg.to_frame(name='Natrium AVG (postoperative)')
natrium_post_sd = natrium_post.groupby(level=0).apply(lambda x: x['V4'].std()).to_frame(name='Natrium SD (postoperative)')
post_op = post_op.merge(natrium_post_avg, left_index=True, right_index=True)
post_op = post_op.merge(natrium_post_sd, left_index=True, right_index=True)


In [ ]:
kalium_post = post_lab[post_lab['V2'] == 'KALIUM;P']
kalium_post['V4'] = pd.to_numeric(kalium_post['V4'], errors='coerce')
kalium_post = kalium_post.dropna(subset=['V4'])
kalium_post_avg = kalium_post.groupby(level=0).apply(lambda x: x['V4'].mean())
kalium_post_avg = kalium_post_avg.to_frame(name='Kalium AVG (postoperative)')
kalium_post_sd = kalium_post.groupby(level=0).apply(lambda x: x['V4'].std()).to_frame(name='Kalium SD (postoperative)')
post_op = post_op.merge(kalium_post_avg, left_index=True, right_index=True)
post_op = post_op.merge(kalium_post_sd, left_index=True, right_index=True)

In [ ]:
cirk = cirk.merge(anaestesi, left_index=True, right_index=True).drop(columns=['Hændelse', 'Hændelsestidspunkt'])

In [ ]:
cirk_post_mask = cirk.groupby(level = 0, sort=False).apply(lambda x: (startstop[x.index[0]][1] < x['Målingsdatotid']))
cirk_post = cirk[cirk_post_mask.values]

In [ ]:
satu_post = cirk_post['Saturation']
satu_post = pd.to_numeric(satu_post, errors='coerce')
satu_post = satu_post.dropna()
satu_post_avg = satu_post.groupby(level=0, sort=False).apply(lambda x: x.mean()).to_frame(name='Saturation AVG (postoperative)')
satu_post_sd = satu_post.groupby(level=0, sort=False).apply(lambda x: x.std()).to_frame(name='Saturation SD (postoperative)')
puls_post = cirk_post['Puls']
puls_post = pd.to_numeric(puls_post, errors='coerce')
puls_post = puls_post.dropna()
puls_post_avg = puls_post.groupby(level=0, sort=False).apply(lambda x: x.mean()).to_frame(name='Puls AVG (postoperative)')
puls_post_sd = puls_post.groupby(level=0, sort=False).apply(lambda x: x.std()).to_frame(name='Puls SD (postoperative)')
post_op = post_op.merge(satu_post_avg, left_index=True, right_index=True)
post_op = post_op.merge(satu_post_sd, left_index=True, right_index=True)
post_op = post_op.merge(puls_post_avg, left_index=True, right_index=True)
post_op = post_op.merge(puls_post_sd, left_index=True, right_index=True)


In [ ]:
dialyse = dialyse.merge(anaestesi, left_index=True, right_index=True).drop(columns=['Hændelse', 'Hændelsestidspunkt'])

In [ ]:
dialyse_post_mask = dialyse.groupby(level = 0, sort=False).apply(lambda x: (startstop[x.index[0]][1] < x['Målingsdatotid']))
dialyse_post = dialyse[dialyse_post_mask.values]

In [ ]:
nyresvigt = list(dialyse_post.index.unique())

In [ ]:
a_bool = []
for i in post_op.index:
    a_bool.append(i in nyresvigt)
sum(a_bool)

In [ ]:
perie_op = perie_op.rename(columns={'Puls AVG(perieoperativ)':'Puls AVG(perioperativ)', 'Puls SD(perieoperative)' : 'Puls SD(perioperative)', 'Saturation SD(perieoperative)' : 'Saturation SD(perioperative)', 'Saturation AVG(perieoperative)' : 'Saturation AVG(perioperative)', 'Total blødning ml (perieoperative)' : 'Total blødning ml (perieoperative)', 'Total urin ml (perieoperative)' : 'Total urin ml (perioperative)'}, inplace=True)

In [ ]:
death_year.to_csv('pre_operative_dataset.csv')
post_op.to_csv('post_operative_dataset.csv')


In [ ]:
perie_op.to_csv('peri_operative_dataset.csv')

In [ ]:
post_op = pd.read_csv('post_operative_dataset.csv', index_col='IDno')

In [ ]:
pre_op = pd.read_csv('pre_operative_dataset.csv', index_col='IDno')

# Predicting mortallity

## Creating dataset containing same variables as EuroSCORE II

In [ ]:
population_no_dup = population[~population.index.duplicated(keep='first')]

In [ ]:
age = population['Alder på operationstidspunkt'][~population.index.duplicated(keep='first')]
sex = population['Køn'][~population.index.duplicated(keep='first')]


In [ ]:
sks_lung = ["DJ40"
,"DJ409"
,"DJ41"
,"DJ410"
,"DJ411"
,"DJ418"
,"DJ42"
,"DJ429"
,"DJ429A"
,"DJ429B"
,"DJ43"
,"DJ430"
,"DJ431"
,"DJ431A"
,"DJ432"
,"DJ438"
,"DJ439"
,"DJ439A"
,"DJ44"
,"DJ440"
,"DJ441"
,"DJ448"
,"DJ448A"
,"DJ448B"
,"DJ448C"
,"DJ449"
,"DJ45"
,"DJ450"
,"DJ450A"
,"DJ451"
,"DJ451A"
,"DJ458"
,"DJ459"
,"DJ46"
,"DJ469"
,"DJ47"
,"DJ479"] 

In [ ]:
euro_df = pd.DataFrame({
    'age': age,
    'sex': sex})

In [ ]:
chronic_lung = pd.DataFrame(problem[problem['Diagnose SKS'].isin(sks_lung)]['Diagnose SKS'])
euro_df['Chronic lung disease'] = 0

for i in euro_df.index: 
    if i in chronic_lung.index:
        euro_df.loc[i, 'Chronic lung disease'] = 1